# **Data Cleaning**

This notebook implements a rule-based data cleaning process based on the findings and recommendations from the exploratory data analysis (EDA).

The EDA revealed that the dataset contains:
- a large number of duplicated rows,
- a significant amount of missing values,
- many clearly invalid and unrealistic values, especially in area-related features and price,
- and strong inconsistencies across several numerical columns.

Based on these findings, this notebook applies a systematic cleaning strategy focused on:
- removing duplicated records,
- removing records with invalid target values,
- replacing physically or logically impossible values with missing values (NaN),
- removing records that do not contain any usable information about property size.

This step intentionally does not perform any imputation, encoding, scaling or feature selection.
Its sole purpose is to remove objectively corrupted data and produce a logically consistent, cleaned dataset that will serve as the input for the preprocessing and modeling pipeline.



## 1. Import Libraries and Load Data

In [28]:
import os
import pandas as pd
import numpy as np

# Show current working directory
print("Current working directory:", os.getcwd())

# If running from notebooks/, go to project root
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
    print("Changed working directory to:", os.getcwd())

# List files to confirm structure
print("Files in project root:", os.listdir("."))
print("Files in data folder:", os.listdir("data"))


Current working directory: c:\projetcs\real-estate-ml-system\notebooks
Changed working directory to: c:\projetcs\real-estate-ml-system
Files in project root: ['.git', '.gitignore', 'api', 'data', 'docker-compose.yml', 'DockerFile', 'documentation', 'kafka', 'LICENSE', 'model', 'notebooks', 'README.md', 'requirements.txt', 'training']
Files in data folder: ['.gitkeep', 'portugal_listings.csv', 'portugal_listings_cleaned.csv']


In [ ]:
df = pd.read_csv("data/portugal_listings.csv")
df.sample(5)

(135536, 25)


C:\Users\jakub\AppData\Local\Temp\ipykernel_33760\2627813385.py:1: DtypeWarning: Columns (9,10,12,13,14,15,16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/portugal_listings.csv")


,Price,District,City,Town,Type,EnergyCertificate,GrossArea,TotalArea,Parking,HasParking,...,Elevator,ElectricCarsCharging,TotalRooms,NumberOfBedrooms,NumberOfWC,ConservationStatus,LivingArea,LotSize,BuiltArea,NumberOfBathrooms
110090,485000.0,Setúbal,Almada,Charneca de Caparica e Sobreda,Land,NC,NaN,NaN,0.0,NaN,...,False,False,NaN,NaN,0.0,NaN,NaN,1583.0,NaN,0.0
127833,200000.0,Lisboa,Cascais,São Domingos de Rana,Apartment,D,NaN,67.0,0.0,NaN,...,False,False,NaN,1.0,1.0,NaN,67.0,NaN,NaN,1.0
56369,59900.0,Santarém,Chamusca,Chamusca e Pinheiro Grande,Store,C,127.0,127.0,0.0,False,...,False,NaN,NaN,NaN,NaN,NaN,116.0,NaN,NaN,NaN
111634,265000.0,Setúbal,Seixal,Fernão Ferro,Apartment,D,NaN,103.0,0.0,NaN,...,False,False,NaN,3.0,0.0,Good condition,100.0,NaN,103.17,2.0
102571,395000.0,Porto,Porto,"Cedofeita, Santo Ildefonso, Sé, Miragaia, São ...",Apartment,E,NaN,131.0,0.0,NaN,...,False,False,3.0,2.0,0.0,Good condition,95.0,NaN,139.30,3.0


In [32]:
df.shape

(135536, 25)

## 2. Remove Duplicated Rows

In [33]:
before = len(df)
df = df.drop_duplicates()
print("Removed duplicates:", before - len(df))
print("Shape after:", df.shape)

Removed duplicates: 8913
Shape after: (126623, 25)


## 3. Clean target variable (price)

### 3.1 Check invalid Prices

In [34]:
df["Price"].isna().sum(), (df["Price"] <= 0).sum()

(np.int64(283), np.int64(0))

### 3.2 Apply Price Cleaning Rules

In [35]:
before = len(df)
df = df[df["Price"].notna()]
df = df[df["Price"] > 0]
print("Removed rows due to invalid Price:", before - len(df))
print("Shape after:", df.shape)

Removed rows due to invalid Price: 283
Shape after: (126340, 25)


## 4. Transform price with logarithm

In [36]:
df["LogPrice"] = np.log1p(df["Price"])
before = len(df)
df = df[np.isfinite(df["LogPrice"])]
print("Removed non-finite LogPrice rows:", before - len(df))

Removed non-finite LogPrice rows: 0


## 5. Clean area features 

### 5.1 Define Area-Related Features

In [37]:
area_cols = ["LivingArea", "TotalArea", "BuiltArea", "LotSize", "GrossArea"]

### 5.2 Stats before Cleaning

In [38]:
df[area_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
LivingArea,97862.0,1424.440794,3.566683e+04,0.0,80.0,118.0,205.0,5.429000e+06
TotalArea,118538.0,551096.694528,1.784448e+08,-7196067.0,91.0,159.0,512.0,6.142007e+10
BuiltArea,24798.0,3175.854571,1.226273e+05,-1.0,103.7,167.0,300.0,1.275000e+07
LotSize,36827.0,57071.542808,5.374487e+06,0.0,261.0,693.0,2948.0,9.923010e+08
GrossArea,25993.0,2930.125957,1.189573e+05,-7.0,100.0,163.0,296.0,1.275000e+07


### 5.3 Apply Area Cleaning Rules:
- values less or equal to 0 are set to NaN
- values greater than 10000 are set to NaN

In [39]:
for col in area_cols:
    df.loc[df[col] <= 0, col] = np.nan
    df.loc[df[col] > 10000, col] = np.nan

### 5.4 Stats after Cleaning

In [40]:
df[area_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
LivingArea,95855.0,319.123843,913.159669,1.000,80.0,116.0,199.0,10000.0
TotalArea,112412.0,678.229477,1475.185341,1.000,90.0,150.0,410.0,10000.0
BuiltArea,24106.0,406.573085,997.463166,0.524,104.0,165.0,290.0,10000.0
LotSize,32428.0,1497.952263,2067.319059,1.000,247.0,580.0,1760.0,10000.0
GrossArea,25208.0,398.638329,984.680231,1.000,101.0,162.0,287.0,10000.0


## 6. Clean rooms and bathrooms 

### 6.1 Define room-related features

In [41]:
room_cols = ["TotalRooms", "NumberOfBedrooms", "NumberOfBathrooms", "NumberOfWC"]

### 6.2 Stats before Cleaning

In [42]:
df[room_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
TotalRooms,68627.0,3.235316,10.828099,0.0,2.0,3.0,4.0,2751.0
NumberOfBedrooms,43699.0,2.672761,1.794965,0.0,2.0,3.0,3.0,21.0
NumberOfBathrooms,119844.0,1.541980,1.737963,-13.0,0.0,1.0,2.0,131.0
NumberOfWC,53131.0,0.432723,1.016968,-15.0,0.0,0.0,1.0,59.0


### 6.3 Apply Rooms and Bathrooms Cleaning Rules:
- values less than 0 are set to NaN
- values greater than 50 are set to NaN

In [43]:
for col in room_cols:
    df.loc[df[col] < 0, col] = np.nan
    df.loc[df[col] > 50, col] = np.nan

### 6.4 Stats after Cleaning

In [44]:
df[room_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
TotalRooms,68610.0,3.180338,2.490436,0.0,2.0,3.0,4.0,49.0
NumberOfBedrooms,43699.0,2.672761,1.794965,0.0,2.0,3.0,3.0,21.0
NumberOfBathrooms,119835.0,1.538324,1.636422,0.0,0.0,1.0,2.0,49.0
NumberOfWC,53129.0,0.431911,0.982454,0.0,0.0,0.0,1.0,40.0


## 7. Clean construction year

- values less than 1800 are set to NaN
- values greater than the current year are set to NaN

In [45]:
df.loc[df["ConstructionYear"] < 1800, "ConstructionYear"] = np.nan
df.loc[df["ConstructionYear"] > 2026, "ConstructionYear"] = np.nan

## 8. Remove Records without Usable Area Information

In [46]:
before = len(df)
df = df.dropna(subset=area_cols, how="all")
print("Removed rows without any area info:", before - len(df))
print("Shape after:", df.shape)


Removed rows without any area info: 5156
Shape after: (121184, 26)


## 9. Final sanity check

In [48]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Price,121184.0,363091.649340,4.005975e+06,1.000000,86500.000000,215000.000000,390000.000000,1.380000e+09
GrossArea,25208.0,398.638329,9.846802e+02,1.000000,101.000000,162.000000,287.000000,1.000000e+04
TotalArea,112412.0,678.229477,1.475185e+03,1.000000,90.000000,150.000000,410.000000,1.000000e+04
Parking,121071.0,0.561299,8.440895e-01,0.000000,0.000000,0.000000,1.000000,3.000000e+00
ConstructionYear,82141.0,1988.934247,2.649677e+01,1900.000000,1972.000000,1994.000000,2008.000000,2.025000e+03
TotalRooms,67630.0,3.213219,2.473666e+00,0.000000,2.000000,3.000000,4.000000,4.900000e+01
NumberOfBedrooms,43195.0,2.697789,1.775005e+00,0.000000,2.000000,3.000000,3.000000,2.100000e+01
NumberOfWC,51120.0,0.447770,9.964878e-01,0.000000,0.000000,0.000000,1.000000,4.000000e+01
LivingArea,95855.0,319.123843,9.131597e+02,1.000000,80.000000,116.000000,199.000000,1.000000e+04
LotSize,32428.0,1497.952263,2.067319e+03,1.000000,247.000000,580.000000,1760.000000,1.000000e+04


In [ ]:
df.isna().mean().sort_values(ascending=False).head(15)

ConservationStatus       0.853009
BuiltArea                0.801079
GrossArea                0.791986
Floor                    0.785516
PublishDate              0.775581
LotSize                  0.732407
NumberOfBedrooms         0.643559
NumberOfWC               0.578162
EnergyEfficiencyLevel    0.503573
Garage                   0.503573
ElectricCarsCharging     0.503573
HasParking               0.496427
TotalRooms               0.441923
ConstructionYear         0.322179
LivingArea               0.209013
dtype: float64

## 10. Save Cleaned Data

In [51]:
df.to_csv("data/portugal_cleaned.csv", index=False)

# **Final Summary**

This notebook implemented a systematic, rule-based data cleaning process based directly on the findings from the exploratory data analysis (EDA).

The raw dataset contained a large number of duplicated rows, invalid target values, physically impossible numerical values and extreme outliers, especially in area-related features and room counts.

As a result of the cleaning process:

- All duplicated records were removed.
- All records with missing or non-positive target values (Price) were removed.
- A log-transformed target variable (LogPrice) was created for stable modeling.
- Area-related features were cleaned by:
  - replacing non-positive values with NaN,
  - removing extreme and clearly invalid values (greater than 10,000 m²).
- Room and bathroom counts were cleaned by removing negative and unrealistic values.
- ConstructionYear values outside a realistic historical range were removed.
- Records without any usable information about property size were removed.

After cleaning:

- The dataset size was reduced from approximately 135k rows to ~130k rows, mostly due to duplicate removal and elimination of invalid records.
- The remaining missing values represent real incompleteness of the data, not corrupted values.
- No imputation, scaling, encoding or feature selection was performed at this stage.

This cleaning step produced a logically consistent and stable dataset that still preserves the full complexity and realism of the original data.

The resulting dataset is now suitable as input for the preprocessing pipeline, where missing value imputation, feature engineering, encoding and scaling will be performed in a fully reproducible and model-ready manner using scikit-learn pipelines.
